# 1. Data Understanding

In [1]:
! wget https://storage.googleapis.com/grizzy-lab/LoanStats_web.csv ./

--2022-06-29 00:32:14--  https://storage.googleapis.com/grizzy-lab/LoanStats_web.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.128, 142.250.159.128, 142.251.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1160243241 (1.1G) [text/csv]
Saving to: ‘LoanStats_web.csv’

LoanStats_web.csv   100%[===================>]   1.08G   157MB/s    in 7.7s    

2022-06-29 00:32:22 (144 MB/s) - ‘LoanStats_web.csv’ saved [1160243241/1160243241]

--2022-06-29 00:32:22--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2022-06-29 00:32:22--
Total wall clock time: 8.4s
Downloaded: 1 files, 1.1G in 7.7s (144 MB/s)


In [2]:
! ls -lh LoanStats_web.csv

-rw-r--r-- 1 root root 1.1G Aug 14  2021 LoanStats_web.csv


In [3]:
! head -5 LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [4]:
! wc -l LoanStats_web.csv

1432493 LoanStats_web.csv


#### นำข้อมูลไปวางไว้ใน HDFS

In [5]:
! hdfs dfs -rm /user/root/LoanStats_web.csv

rm: `/user/root/LoanStats_web.csv': No such file or directory


In [6]:
! hdfs dfs -put LoanStats_web.csv /user/root/

In [7]:
! hdfs dfs -ls /user/root/

Found 2 items
drwxr-xr-x   - root hadoop          0 2022-06-29 00:28 /user/root/.sparkStaging
-rw-r--r--   2 root hadoop 1160243241 2022-06-29 00:32 /user/root/LoanStats_web.csv


In [8]:
raw_df = spark.read.format('csv').option('header','true')\
.option('mode','DROPMALFORMED').load('/user/root/LoanStats_web.csv')

#### นับจำนวน Attributes

In [9]:
len(raw_df.columns)

144

In [10]:
raw_df.count()

1432466

In [11]:
selectedAttr_df = raw_df.select("loan_amnt","term","int_rate"\
                                ,"installment","grade","emp_length",\
                           "home_ownership","annual_inc"\
                                ,"verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs"\
                                ,"earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec"\
                                ,"revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [12]:
len(selectedAttr_df.columns)

22

In [13]:
! pip install pandas

In [14]:
selectedAttr_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432440,15370.388358325654,9646.026272413876,1000,9975
term,1432440,None,None,36 months,60 months
int_rate,1432440,None,None,5.31%,30.99%
installment,1432440,456.6878153569824,281.71344223131814,100.01,999.98
grade,1432440,None,None,A,G
emp_length,1432440,None,None,1 year,n/a
home_ownership,1432440,None,None,ANY,RENT
annual_inc,1432440,81034.5829329675,134183.35696714345,0,99999.84
verification_status,1432440,None,None,Not Verified,Verified


# 2. Data Preparation

#### ลบทิ้งทั้งบรรทัด หาก attributes ใดมีค่า null

In [15]:
noMissing_df = selectedAttr_df.dropna(how='any')

In [16]:
noMissing_df.count()

1429645

#### แสดง Schema ปรับปรุงข้อมูลต่อไป

In [17]:
noMissing_df.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [18]:
fitmem_noMissing_df = noMissing_df.repartition(60)

In [19]:
cached_noMissing_df = fitmem_noMissing_df.cache()

In [20]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

#### ประกาศ Python & Spark Function สำหรับเอาเครื่องหมาย % ออก

In [21]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [22]:
removepercentsign = udf(lambda x: f_removepercentsign(x)\
                        ,StringType())

#### ประกาศ Pythonn & Spark Function สำหรับเอาเครื่องหมาย - ออก

In [23]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [24]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [25]:
crunched_df = cached_noMissing_df.\
withColumn('revol_util',removepercentsign(cached_noMissing_df['revol_util'])\
           .cast(DoubleType())).\
withColumn('int_rate',removepercentsign(cached_noMissing_df['int_rate'])\
           .cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(cached_noMissing_df['earliest_cr_line'])\
           .cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(cached_noMissing_df['last_credit_pull_d'])\
           .cast(StringType())).\
withColumn('dti',cached_noMissing_df['dti'].cast(DoubleType())).\
withColumn('loan_amnt',cached_noMissing_df['loan_amnt'].cast(DoubleType())).\
withColumn('annual_inc',cached_noMissing_df['annual_inc'].cast(DoubleType()))

In [26]:
crunched_df.groupBy(['revol_util','int_rate'\
                     ,'earliest_cr_line','last_credit_pull_d']).count().show()

+----------+--------+----------------+------------------+-----+
|revol_util|int_rate|earliest_cr_line|last_credit_pull_d|count|
+----------+--------+----------------+------------------+-----+
|      24.2|   11.44|             Aug|               Oct|    4|
|      77.3|   14.99|             Sep|               Sep|    4|
|      39.3|   12.74|             May|               Sep|    4|
|      24.2|   13.49|             Feb|               Jun|    1|
|      62.4|   19.99|             Sep|               Jul|    1|
|      59.1|   13.99|             Oct|               Jul|    4|
|      18.5|    6.08|             Jun|               Oct|    2|
|      27.1|    7.46|             Apr|               Oct|    1|
|      41.4|    6.72|             Jun|               Apr|    1|
|      18.7|   17.47|             Feb|               Oct|    1|
|      20.2|    7.35|             Aug|               Apr|    1|
|      45.0|    7.89|             Nov|               Aug|    1|
|       3.4|   10.75|             Aug|  

In [27]:
crunched_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



#### ประกาศ Python & Spark สำหรับลด Scale ของข้อมูลด้วยเทคนิค Max-Min Normalization

In [28]:
from pyspark.sql.functions import *
max_annual_inc = crunched_df.select(max('annual_inc')).collect()[0][0]
min_annual_inc = crunched_df.select(min('annual_inc')).collect()[0][0]

In [29]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [30]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [31]:
max_loan_amnt = crunched_df.select(max('loan_amnt')).collect()[0][0]
min_loan_amnt = crunched_df.select(min('loan_amnt')).collect()[0][0]

In [32]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [33]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [34]:
normalized_df = crunched_df.withColumn('loan_amnt'\
                ,normalized_loan_amnt(crunched_df['loan_amnt'])).\
withColumn('annual_inc',normalized_annual_inc(crunched_df['annual_inc']))

In [35]:
normalized_df.select(['loan_amnt','annual_inc']).describe().show()

+-------+-------------------+--------------------+
|summary|          loan_amnt|          annual_inc|
+-------+-------------------+--------------------+
|  count|            1429645|             1429645|
|   mean|0.36837690520448685|0.001329699400788...|
| stddev| 0.2472269581024671| 0.00220036950900334|
|    min|                0.0|                 0.0|
|    max|                1.0|                 1.0|
+-------+-------------------+--------------------+



In [36]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



#### แสดงให้เห็นว่ามี Class อื่นๆ ที่ไม่เกี่ยวข้องกับการ Make Prediction อยู่ด้วย

In [37]:
normalized_df.groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|582923|
|           Default|  1416|
|   In Grace Period|  6125|
|       Charged Off|152817|
|Late (31-120 days)| 19405|
|           Current|661982|
| Late (16-30 days)|  4977|
+------------------+------+



#### เลือกมาเฉพาะ Row ที่มี Loan Status เป็น Fully Paid และ Charged Off

In [38]:
from pyspark.sql.functions import col

In [39]:
normalized_filtered_df = normalized_df.\
filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [40]:
normalized_filtered_df.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|582923|
|Charged Off|152817|
+-----------+------+



In [41]:
normalized_filtered_df.count()

735740

In [42]:
final_df = normalized_filtered_df.dropna(how='any')

In [43]:
final_df.count()

735740

In [44]:
final_df.select(['loan_amnt','int_rate'\
                 ,'dti','home_ownership','grade']).\
show()

+-------------------+--------+-----+--------------+-----+
|          loan_amnt|int_rate|  dti|home_ownership|grade|
+-------------------+--------+-----+--------------+-----+
| 0.2564102564102564|   13.99| 5.49|          RENT|    C|
|0.38461538461538464|   17.99|12.61|          RENT|    D|
|0.23076923076923078|   18.99|28.38|          RENT|    D|
|0.20512820512820512|   14.99|20.88|      MORTGAGE|    C|
| 0.1282051282051282|   11.39|17.94|           OWN|    B|
|0.20512820512820512|    8.24|19.07|      MORTGAGE|    B|
|0.10256410256410256|   13.49|11.33|      MORTGAGE|    C|
| 0.3006410256410256|   16.99| 18.8|          RENT|    D|
|0.38461538461538464|   13.49|13.66|          RENT|    C|
|0.39487179487179486|   15.99| 20.3|      MORTGAGE|    C|
| 0.1282051282051282|   12.79|17.69|          RENT|    C|
|0.28205128205128205|   11.39| 23.2|          RENT|    B|
|0.48717948717948717|   13.49|24.54|          RENT|    C|
| 0.1282051282051282|   11.49|25.78|          RENT|    B|
|0.48717948717

# 3. Data Modeling

In [45]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, \
VectorAssembler, OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, \
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, \
GBTClassifier, NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

## 3.1 Transformation categorical variable to numerical one.

In [46]:
labelIndexer = StringIndexer(inputCol='loan_status',\
                             outputCol='indexedLabel')

In [47]:
homeIndexer = StringIndexer(inputCol='home_ownership',\
                            outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,\
                                  inputCol='homeIndexed',\
                                  outputCol='homeVec')

In [48]:
gradeIndexer = StringIndexer(inputCol='grade',\
                             outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,\
                                   inputCol='gradeIndexed',\
                                   outputCol='gradeVec')

### 3.2 Combines selected columns into a single vector column.

In [49]:
featureAssembler = VectorAssembler(inputCols=['loan_amnt',\
                    'int_rate','dti','homeVec','gradeVec'],\
                                   outputCol='***features')

### 3.3 Define an algorithm.

In [50]:
dt = DecisionTreeClassifier(featuresCol='***features',\
                            labelCol='indexedLabel')

### 3.4 Setting up the pipeline.

In [51]:
pipeline_dt = Pipeline().setStages([gradeIndexer,\
                                    gradeOneHotEncoder,\
                                    homeIndexer,\
                                    homeOneHotEncoder,\
                        labelIndexer,featureAssembler,dt])

In [52]:
training_df , test_df = final_df.randomSplit([0.6,0.4])

### 3.5 Launch the pipeline and get a model.

In [53]:
model1 = pipeline_dt.fit(training_df)

# 4. Evaluation of Models

In [54]:
result_df = model1.transform(test_df)

In [55]:
result_df

DataFrame[loan_amnt: double, term: string, int_rate: double, installment: string, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: string, pub_rec: string, revol_bal: string, revol_util: double, total_acc: string, last_credit_pull_d: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, indexedLabel: double, ***features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [56]:
test_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,293980,0.34598065715267584,0.23669331767664506,0.0,1.0
term,293980,None,None,36 months,60 months
int_rate,293980,13.194942921287042,5.149902546057371,5.31,30.99
installment,293980,446.2463027416822,282.57666527788723,100.01,999.98
grade,293980,None,None,A,G
emp_length,293980,None,None,1 year,n/a
home_ownership,293980,None,None,ANY,RENT
annual_inc,293980,0.0013060134865804428,0.0013255534003471844,0.0,0.15655737704918032
verification_status,293980,None,None,Not Verified,Verified


In [57]:
result_df.select('loan_status',\
                 'indexedLabel','prediction')\
.filter(result_df.loan_status == 'Fully Paid').show(20)

+-----------+------------+----------+
|loan_status|indexedLabel|prediction|
+-----------+------------+----------+
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
+-----------+------------+----------+
only showing top 20 rows



In [58]:
result_df.select('loan_status','indexedLabel'\
                 ,'prediction')\
.filter(result_df.loan_status == 'Charged Off').show(20)

+-----------+------------+----------+
|loan_status|indexedLabel|prediction|
+-----------+------------+----------+
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
+-----------+------------+----------+
only showing top 20 rows



In [59]:
result_df.groupBy(['indexedLabel','prediction']).count().show()

+------------+----------+------+
|indexedLabel|prediction| count|
+------------+----------+------+
|         1.0|       1.0|  4148|
|         0.0|       1.0|  3958|
|         1.0|       0.0| 56687|
|         0.0|       0.0|229187|
+------------+----------+------+



In [60]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', \
                                                 metricName='accuracy')
print(evaluator_RF.evaluate(result_df)*100)

79.37104564936391


In [61]:
sc

<SparkContext master=yarn appName=PySparkShell>